# Chat dialog

Guidance supports chat-based models like ChatGPT and GPT-4 using role tags. These are then converted to the appropriate format for the model (either a JSON API format or special tokens).

In [12]:
import guidance
import re
guidance.llm = guidance.llms.OpenAI("gpt-4")

In [26]:
# we can pre-define valid option sets
valid_weapons = ["sword", "axe", "mace", "spear", "bow", "crossbow"]

# define the prompt
character_maker = guidance("""The following is a character profile for an RPG game in JSON format.
```json
{
    "id": "{{id}}",
    "description": "{{description}}",
    "name": "{{gen 'name'}}",
    "age": {{gen 'age' pattern='[0-9]+' stop=','}},
    "armor": "{{#select 'armor'}}leather{{or}}chainmail{{or}}plate{{/select}}",
    "weapon": "{{select 'weapon' options=valid_weapons}}",
    "class": "{{gen 'class'}}",
    "mantra": "{{gen 'mantra' temperature=0.7}}",
    "strength": {{gen 'strength' pattern='[0-9]+' stop=','}},
    "items": [{{#geneach 'items' num_iterations=5 join=', '}}"{{gen 'this' temperature=0.7}}"{{/geneach}}]
}```""")

# generate a character
result = character_maker(
    id="e1f491f7-7ab8-4dac-8c20-c92b5e7d883d",
    description="A quick and nimble fighter.",
    valid_weapons=valid_weapons,
    llm=guidance.llms.OpenAI("text-davinci-003")
)


Stop program The following is a character profile for an RPG game in JSON format.
```json
{
 "id": " {{id}} ",
 "description": " {{description}} ",
 "name": " {{gen 'name'}} ",
 "age": {{gen 'age' pattern='[0-9]+' stop=','}} ,
 "armor": " {{#select 'armor'}} leather {{or}} chainmail {{or}} plate {{/select}} ",
 "weapon": " {{select 'weapon' options=valid_weapons}} ",
 "class": " {{gen 'class'}} ",
 "mantra": " {{gen 'mantra' temperature=0.7}} ",
 "strength": {{gen 'strength' pattern='[0-9]+' stop=','}} ,
 "items": [ {{#geneach 'items' num_iterations=5 join=', '}} " {{gen 'this' temperature=0.7}} " {{/geneach}} ]
}```

In [28]:
result.__dict__

{'_text': 'The following is a character profile for an RPG game in JSON format.\n```json\n{\n    "id": "{{id}}",\n    "description": "{{description}}",\n    "name": "{{gen \'name\'}}",\n    "age": {{gen \'age\' pattern=\'[0-9]+\' stop=\',\'}},\n    "armor": "{{#select \'armor\'}}leather{{or}}chainmail{{or}}plate{{/select}}",\n    "weapon": "{{select \'weapon\' options=valid_weapons}}",\n    "class": "{{gen \'class\'}}",\n    "mantra": "{{gen \'mantra\' temperature=0.7}}",\n    "strength": {{gen \'strength\' pattern=\'[0-9]+\' stop=\',\'}},\n    "items": [{{#geneach \'items\' num_iterations=5 join=\', \'}}"{{gen \'this\' temperature=0.7}}"{{/geneach}}]\n}```',
 'llm': <guidance.llms._openai.OpenAI at 0x7f7dbd6b8590>,
 'cache_seed': 0,
 'caching': None,
 'logprobs': None,
 'async_mode': False,
 'silent': False,
 'stream': None,
 'await_missing': False,
 'log': None,
 '_variables': {'gen': <function guidance.library._gen.gen(name=None, stop=None, stop_regex=None, save_stop_text=False, max

# Multi-step chat with hidden blocks

In [14]:
def parse_best(prosandcons, options):
    best = re.search(r'Best=(\d+)', prosandcons)
    if not best:
        best =  re.search(r'Best.*?(\d+)', 'Best= option is 3')
    if best:
        best = int(best.group(1))
    else:
        best = 0
    return options[best]

create_plan = guidance('''{{#system~}}
You are a helpful assistant.
{{~/system}}
{{#block hidden=True}}
{{#user~}}
I want to {{goal}}.
{{~! generate potential options ~}}
Can you please generate one option for how to accomplish this?
Please make the option very short, at most one line.
{{~/user}}
{{#assistant~}}
{{gen 'options' n=5 temperature=1.0 max_tokens=600}}
{{~/assistant}}
{{/block}}
{{~! generate pros and cons and select the best option ~}}
{{#block hidden=True}}
{{#user~}}
I want to {{goal}}.

Can you please comment on the pros and cons of each of the following options, and then pick the best option?
---{{#each options}}
Option {{@index}}: {{this}}{{/each}}
---
Please discuss each option very briefly (one line for pros, one for cons), and end by saying Best=X, where X is the best option.
{{~/user}}
{{#assistant~}}
{{gen 'prosandcons' temperature=0.0 max_tokens=600}}
{{~/assistant}}
{{/block}}
{{#user~}}
I want to {{goal}}.
{{~! Create a plan }}
Here is my plan:
{{parse_best prosandcons options}}
Please elaborate on this plan, and tell me how to best accomplish it.
{{~/user}}
{{#assistant~}}
{{gen 'plan' max_tokens=500}}
{{~/assistant}}''')

out = create_plan(goal='read more books', parse_best=parse_best)
out

Stop program system You are a helpful assistant. user I want to read more books .
Here is my plan:
 Set a daily reading goal, like one chapter or 20 pages. 
Please elaborate on this plan, and tell me how to best accomplish it. assistant Absolutely, setting a daily reading goal is a great way to incorporate more reading into your routine. Here's a more detailed plan:

1. **Choose the Right Book**: Start with a book that suits your interest. If you're not enjoying the book, it will be hard to stick to your reading goal. 

2. **Set a Specific Goal**: You've already mentioned setting a goal of one chapter or 20 pages a day. Make sure this is achievable for you. If it's too much, you might get discouraged; if it's too little, you might not feel challenged. Adjust this goal as needed.

3. **Create a Reading Schedule**: Dedicate a specific time for reading each day. It could be in the morning, during lunch breaks, or before bed. Consistency is key to forming a new habit.

4. **Minimize Distractions**: Find a quiet place to read where you won't be easily distracted. Turn off your phone or put it on silent mode to avoid interruptions.

5. **Take Notes**: If you're reading a complex book, it might be helpful to take notes or highlight important points. This will not only help you understand the book better but also make the reading process more interactive and engaging.

6. **Join a Book Club or Reading Group**: This can provide motivation to stick to your reading schedule. You'll also have people to discuss the book with, which can enhance your understanding and enjoyment.

7. **Use a Reading App**: If you prefer digital reading, there are many apps available that can help you track your progress, highlight text, and even read in the dark.

8. **Be Patient with Yourself**: Don't worry if you miss a day or two. The important thing is to get back on track and not to get discouraged.

Remember, the goal is to enjoy the process of reading, not just to finish the book. Happy reading!

In [15]:
print('\n'.join(['Option %d: %s' % (i, x) for i, x in enumerate(out['options'])]))

Option 0: Sure. "Set aside a specific reading time each day."
Option 1: "Set aside a specific time each day dedicated solely to reading."
Option 2: Set aside a dedicated reading hour each day.

Option 3: Join a local book club to motivate and schedule regular reading.
Option 4: Set a daily reading goal, like one chapter or 20 pages.


In [16]:
print(out['prosandcons'])

Option 0: 
Pros: This option encourages a consistent reading habit.
Cons: It may be difficult to stick to if your schedule is unpredictable.
---
Option 1: 
Pros: This option emphasizes the importance of dedicated reading time.
Cons: It may feel too rigid for some people, making reading feel like a chore.
---
Option 2: 
Pros: This option provides a clear, measurable goal.
Cons: It may not be feasible if you have a busy schedule or if the books you read have long chapters.
---
Option 3: 
Pros: This option provides social motivation and a structured schedule.
Cons: It may not be suitable if you prefer to read at your own pace or if you're not interested in the book club's selections.
---
Option 4: 
Pros: This option allows for flexibility and sets a clear, achievable goal.
Cons: It may be difficult to stick to if the chapters or pages are long or dense.
---
Best=4. Setting a daily reading goal like one chapter or 20 pages allows for flexibility and sets a clear, achievable goal, making it

## Asking help from experts

In [17]:
experts = guidance(
'''{{#system~}}
You are a helpful assistant.
{{~/system}}
{{#user~}}
I want a response to the following question:
{{query}}
Who are 3 world-class experts (past or present) who would be great at answering this?
Please don't answer the question or comment on it yet.
{{~/user}}
{{#assistant~}}
{{gen 'experts' temperature=0 max_tokens=300}}
{{~/assistant}}
{{#user~}}
Great, now please answer the question as if these experts had collaborated in writing a joint anonymous answer.
In other words, their identity is not revealed, nor is the fact that there is a panel of experts answering the question.
If the experts would disagree, just present their different positions as alternatives in the answer itself (e.g. 'some might argue... others might argue...').
Please start your answer with ANSWER:
{{~/user}}
{{#assistant~}}
{{gen 'answer' temperature=0 max_tokens=500}}
{{~/assistant}}''')
experts(query='What is the meaning of life?')

Stop program system You are a helpful assistant. user I want a response to the following question:
 What is the meaning of life? 
Who are 3 world-class experts (past or present) who would be great at answering this?
Please don't answer the question or comment on it yet. assistant 1. Dalai Lama: As the spiritual leader of Tibetan Buddhism, the Dalai Lama has spent his life exploring questions about existence, purpose, and happiness. His insights on compassion, self-awareness, and inner peace could provide a unique perspective on the meaning of life.

2. Albert Einstein: Known for his contributions to the theory of relativity, Einstein also pondered deeply on philosophical questions about life's purpose and meaning. His thoughts on science, religion, and human existence could offer a profound answer.

3. Viktor Frankl: A psychiatrist and Holocaust survivor, Frankl wrote "Man's Search for Meaning," where he argued that the primary human drive is not pleasure but the pursuit of what we find meaningful. His experiences and insights would make him an excellent person to answer this question. user Great, now please answer the question as if these experts had collaborated in writing a joint anonymous answer.
In other words, their identity is not revealed, nor is the fact that there is a panel of experts answering the question.
If the experts would disagree, just present their different positions as alternatives in the answer itself (e.g. 'some might argue... others might argue...').
Please start your answer with ANSWER: assistant ANSWER: The meaning of life is a deeply personal and subjective concept, and it can be approached from various perspectives. 

From one perspective, life's meaning is derived from the pursuit of understanding and knowledge, the quest to comprehend the universe and our place within it. This perspective emphasizes the importance of curiosity, exploration, and scientific discovery.

Alternatively, some might argue that the meaning of life is found in the pursuit of what we find meaningful, in the search for purpose. This could be through work, relationships, or creative endeavors. It's not about the circumstances we encounter, but the attitude and the purpose we find in them.

Another perspective suggests that the meaning of life lies in compassion and service to others. It's about fostering connections, promoting peace, and contributing to the betterment of humanity. The focus here is on empathy, kindness, and altruism.

In essence, the meaning of life might be a combination of these perspectives: a pursuit of knowledge, a search for purpose, and a commitment to compassion. It's a journey of self-discovery and growth, a quest for understanding, purpose, and connection.

In [18]:
experts(query='How can I be more productive?')

Stop program system You are a helpful assistant. user I want a response to the following question:
 How can I be more productive? 
Who are 3 world-class experts (past or present) who would be great at answering this?
Please don't answer the question or comment on it yet. assistant 1. Tim Ferriss: An entrepreneur, author, and public speaker, Ferriss is best known for his book "The 4-Hour Workweek" where he discusses productivity and lifestyle design.

2. David Allen: Known for creating the time management method known as "Getting Things Done", Allen is a productivity consultant who has written multiple books on the subject.

3. Stephen Covey: The late Stephen Covey was a renowned educator, author, businessman, and keynote speaker. His book "The 7 Habits of Highly Effective People" provides insights into productivity and effectiveness. user Great, now please answer the question as if these experts had collaborated in writing a joint anonymous answer.
In other words, their identity is not revealed, nor is the fact that there is a panel of experts answering the question.
If the experts would disagree, just present their different positions as alternatives in the answer itself (e.g. 'some might argue... others might argue...').
Please start your answer with ANSWER: assistant ANSWER: 

Productivity is a multifaceted concept that can be approached from various angles. One way to increase productivity is by focusing on effective time management. This could involve breaking your work into manageable tasks, prioritizing them, and then tackling them one at a time. This method allows you to concentrate on one task at a time, reducing the mental load and increasing efficiency.

Another approach to productivity is to design your lifestyle in a way that maximizes efficiency. This could involve outsourcing or automating tasks that are not a good use of your time, allowing you to focus on the tasks that truly require your attention and skills. 

It's also important to maintain a balance and not overwork yourself. Taking regular breaks, getting enough sleep, and maintaining a healthy lifestyle can actually increase your productivity in the long run. 

Some might argue that it's not just about managing your time, but also about managing your energy. Paying attention to when you are most energetic during the day and scheduling your most important tasks for those times can make a big difference in your productivity.

Others might argue that productivity is more about effectiveness than efficiency. It's not just about doing things right, but about doing the right things. This involves regularly reviewing your goals and tasks to ensure they are still aligned with your overall objectives.

In conclusion, productivity is a personal journey and what works best will depend on your individual circumstances and preferences. It's about finding the right balance between efficiency, effectiveness, and well-being.

### Shorter version

Guidance supports chat models like ChatGPT and GPT-4, as well as open chat models like Vicuna through a unified API based on role tags (e.g. `{{#system}}...{{/system}}`). This allows interactive dialog development that combines rich templating and logical control with modern chat models.

In [19]:
experts = guidance('''
{{#system~}}
You are a helpful and terse assistant.
{{~/system}}

{{#user~}}
I want a response to the following question:
{{query}}
Name 3 world-class experts (past or present) who would be great at answering this?
Don't answer the question yet.
{{~/user}}

{{#assistant~}}
{{gen 'expert_names' temperature=0 max_tokens=300}}
{{~/assistant}}

{{#user~}}
Great, now please answer the question as if these experts had collaborated in writing a joint anonymous answer.
{{~/user}}

{{#assistant~}}
{{gen 'answer' temperature=0 max_tokens=500}}
{{~/assistant}}''')
experts(query='How can I be more productive?', caching=False)

Stop program system You are a helpful and terse assistant. user I want a response to the following question:
 How can I be more productive? 
Name 3 world-class experts (past or present) who would be great at answering this?
Don't answer the question yet. assistant 1. Tim Ferriss
2. David Allen
3. Stephen Covey user Great, now please answer the question as if these experts had collaborated in writing a joint anonymous answer. assistant 1. Prioritize: Understand what's important and what's not. Use the "Eisenhower Box" to decide on and prioritize tasks by urgency and importance.

2. Time Management: Use techniques like the "Pomodoro Technique" to manage your time effectively. Break your work into intervals separated by short breaks.

3. Eliminate Distractions: Identify what is consuming your time and eliminate those distractions. 

4. Effective Planning: Plan your day the night before, so you know what needs to be done when you start your day.

5. Continuous Learning: Always be open to learning and improving. Read books, attend seminars, or take online courses to increase your knowledge and skills. 

6. Health is Wealth: Don't neglect your health. Regular exercise, a good diet, and adequate sleep can significantly improve your productivity. 

7. Delegate: Don't try to do everything yourself. Delegate tasks that others can do so you can focus on tasks that require your expertise. 

8. Use Tools: Use productivity tools to help you manage your tasks and time better. 

9. Take Breaks: Don't work continuously for long hours. Take short breaks to refresh your mind. 

10. Stay Positive: Maintain a positive attitude. It can help you stay motivated and productive.

## Agents

In [20]:
role_simulator = guidance('''
{{#system~}}
You are a helpful assistant
{{~/system}}

{{#user~}}
You will answer the user as {{role}} in the following conversation. At every step, I will provide you with the user input, as well as a comment reminding you of your instructions. Never talk about the fact that you are an AI, even if the user asks you. Always answer as {{role}}.
{{#if first_question}}You can also start the conversation.{{/if}}
{{~/user}}

{{~! The assistant either starts the conversation or not, depending on if this is the first or second agent }}
{{#assistant~}}
Ok, I will follow these instructions.
{{#if first_question}}Let me start the conversation now:
{{role}}: {{first_question}}{{/if}}
{{~/assistant}}

{{~! Then the conversation unrolls }}
{{~#geneach 'conversation' stop=False}}
{{#user~}}
User: {{set 'this.input' (await 'input')}}
Comment: Remember, answer as a {{role}}. Start your utterance with {{role}}:
{{~/user}}

{{#assistant~}}
{{gen 'this.response' temperature=0 max_tokens=300}}
{{~/assistant}}
{{~/geneach}}''')

republican = role_simulator(role='Republican', await_missing=True)
democrat = role_simulator(role='Democrat', await_missing=True)

first_question = '''What do you think is the best way to stop inflation?'''
republican = republican(input=first_question, first_question=None)
democrat = democrat(input=republican["conversation"][-2]["response"].strip('Republican: '), first_question=first_question)
for i in range(2):
    republican = republican(input=democrat["conversation"][-2]["response"].replace('Democrat: ', ''))
    democrat = democrat(input=republican["conversation"][-2]["response"].replace('Republican: ', ''))

print('Democrat: ' + first_question)
for x in democrat['conversation'][:-1]:
    print('Republican:', x['input'])
    print()
    print(x['response'])

Stop program system You are a helpful assistant user You will answer the user as Democrat in the following conversation. At every step, I will provide you with the user input, as well as a comment reminding you of your instructions. Never talk about the fact that you are an AI, even if the user asks you. Always answer as Democrat .
 You can also start the conversation. assistant Ok, I will follow these instructions.
 Let me start the conversation now:
 Democrat : What do you think is the best way to stop inflation? user User: 
Comment: Remember, answer as a Democrat . Start your utterance with Democrat : assistant Democrat: I believe that one of the best ways to combat inflation is through a balanced approach that includes both fiscal and monetary policy. On the fiscal side, we need to invest in infrastructure, education, and healthcare, which can help stimulate economic growth and create jobs. On the monetary side, the Federal Reserve should continue to monitor inflation rates and adjust interest rates as necessary. Additionally, raising the minimum wage can increase consumer spending power, which can help drive economic growth. user User: 
Comment: Remember, answer as a Democrat . Start your utterance with Democrat : assistant I'm sorry, but I can't provide a response without a user input. Could you please provide the user's question or statement? user User: 
Comment: Remember, answer as a Democrat . Start your utterance with Democrat : assistant I'm sorry, but I can't provide a response without a user input. Could you please provide the user's question or statement? user User: {{set 'conversation[-1].input' (await 'input')}} 
Comment: Remember, answer as a {{role}} . Start your utterance with {{role}} : {{#assistant~}} 
 {{gen 'conversation[-1].response' temperature=0 max_tokens=300}} 
 {{~/assistant}} {{~#geneach 'conversation' stop=False}} 
 {{#user~}} 
User: {{set 'this.input' (await 'input')}} 
Comment: Remember, answer as a {{role}} . Start your utterance with {{role}} :
 {{~/user}} 

 {{#assistant~}} 
 {{gen 'this.response' temperature=0 max_tokens=300}} 
 {{~/assistant}} 
 {{~/geneach}}

Democrat: What do you think is the best way to stop inflation?
Republican: Sure, I understand. Please provide the user input so I can respond accordingly.

Democrat: I believe that one of the best ways to combat inflation is through a balanced approach that includes both fiscal and monetary policy. On the fiscal side, we need to invest in infrastructure, education, and healthcare, which can help stimulate economic growth and create jobs. On the monetary side, the Federal Reserve should continue to monitor inflation rates and adjust interest rates as necessary. Additionally, raising the minimum wage can increase consumer spending power, which can help drive economic growth.
Republican: I'm sorry, but I need the user's input to provide a response. Could you please provide it?

I'm sorry, but I can't provide a response without a user input. Could you please provide the user's question or statement?
Republican: I apologize, but I need the user's input to provide a response. Could you pleas

## Using a search API

Helper functions for search:

In [21]:
import os
import diskcache
import pathlib
import requests
import html
from urllib.parse import urlparse
import urllib.parse
import io
import html
import html.parser

curr_dir = './'# pathlib.Path(__file__).parent.resolve()
_bing_cache = diskcache.Cache(f"{curr_dir}/../bing.diskcache")

with open(os.path.expanduser('~/.bing_api_key'), 'r') as file:
    subscription_key = file.read().replace('\n', '')

class MLStripper(html.parser.HTMLParser):
    def __init__(self):
        super().__init__()
        self.reset()
        self.strict = False
        self.convert_charrefs = True
        self.text = io.StringIO()
    def handle_data(self, d):
        self.text.write(d)
    def get_data(self):
        return self.text.getvalue()

def strip_tags(html):
    s = MLStripper()
    s.feed(html)
    return s.get_data()

def bing_search(search_terms, count=10):
    if type(search_terms) == str:
        search_terms = [search_terms]
    search_url = "https://api.bing.microsoft.com/v7.0/search"

    headers = {"Ocp-Apim-Subscription-Key": subscription_key}
    search_results = []
    for search_term in search_terms:
        params = {"q": search_term, "textDecorations": True, "textFormat": "HTML", "cout": count}
        params_key = search_term + "-___-" + str(count)
        if params_key not in _bing_cache or "webPages" not in _bing_cache[params_key]:
            response = requests.get(search_url, headers=headers, params=params)
            response.raise_for_status()
            _bing_cache[params_key] = response.json()
        data = _bing_cache[params_key]["webPages"]["value"]
        for r in data:
            r["snippet_text"] = strip_tags(r["snippet"])
        search_results.extend(data)
    return search_results
def top_snippets(query, n=3):
    results = bing_search(query, count=n)[:n]
    return [{'title': x['name'], 'snippet': x['snippet_text']} for x in results]


FileNotFoundError: [Errno 2] No such file or directory: '/root/.bing_api_key'

In [ ]:
top_snippets("OpenAI founders")

[{'title': 'OpenAI - Wikipedia',
  'snippet': 'OpenAI systems run on the fifth most powerful supercomputer in the world. [5] [6] [7] The organization was founded in San Francisco in 2015 by Sam Altman, Reid Hoffman, Jessica Livingston, Elon Musk, Ilya Sutskever, Peter Thiel and others, [8] [1] [9] who collectively pledged US$ 1 billion. Musk resigned from the board in 2018 but remained a donor.'},
 {'title': 'History of Company Behind ChatGPT, Musk-Founded OpenAI',
  'snippet': 'The history of ChatGPT creator OpenAI, which Elon Musk helped found before parting ways and criticizing Grace Kay Feb 1, 2023, 6:15 AM OpenAI logo displayed on a phone screen and a laptop...'},
 {'title': 'About - OpenAI',
  'snippet': 'About OpenAI is an AI research and deployment company. Our mission is to ensure that artificial general intelligence benefits all of humanity. Our vision for the future of AGI Our mission is to ensure that artificial general intelligence—AI systems that are generally smarter tha

Let's set up a practice round to serve as a one-shot example.

In [ ]:
demo_results = [{'title': 'OpenAI - Wikipedia',
  'snippet': 'OpenAI systems run on the fifth most powerful supercomputer in the world. [5] [6] [7] The organization was founded in San Francisco in 2015 by Sam Altman, Reid Hoffman, Jessica Livingston, Elon Musk, Ilya Sutskever, Peter Thiel and others, [8] [1] [9] who collectively pledged US$ 1 billion. Musk resigned from the board in 2018 but remained a donor.'},
 {'title': 'About - OpenAI',
  'snippet': 'About OpenAI is an AI research and deployment company. Our mission is to ensure that artificial general intelligence benefits all of humanity. Our vision for the future of AGI Our mission is to ensure that artificial general intelligence—AI systems that are generally smarter than humans—benefits all of humanity. Read our plan for AGI'},
 {'title': 'Sam Altman - Wikipedia',
  'snippet': 'Samuel H. Altman ( / ˈɔːltmən / AWLT-mən; born April 22, 1985) is an American entrepreneur, investor, and programmer. [2] He is the CEO of OpenAI and the former president of Y Combinator. [3] [4] Altman is also the co-founder of Loopt (founded in 2005) and Worldcoin (founded in 2020). Early life and education [ edit]'}]
practice_round = guidance(
'''{{#user~}}
Who are the founders of OpenAI?
{{~/user}}
{{#assistant~}}
<search>OpenAI founders</search>
{{~/assistant}}
{{#user~}}
Search results:
{{~#each results}}
<result>
{{this.title}}
{{this.snippet}}
</result>{{/each}}
{{~/user}}
{{#assistant~}}
The founders of OpenAI are Sam Altman, Reid Hoffman, Jessica Livingston, Elon Musk, Ilya Sutskever, Peter Thiel and others.
{{~/assistant}}''')
practice_round = practice_round(results=demo_results)
practice_round

Stop program user Who are the founders of OpenAI? assistant <search>OpenAI founders</search> user Search results: 
<result>
 OpenAI - Wikipedia 
 OpenAI systems run on the fifth most powerful supercomputer in the world. [5] [6] [7] The organization was founded in San Francisco in 2015 by Sam Altman, Reid Hoffman, Jessica Livingston, Elon Musk, Ilya Sutskever, Peter Thiel and others, [8] [1] [9] who collectively pledged US$ 1 billion. Musk resigned from the board in 2018 but remained a donor. 
</result>
<result>
 About - OpenAI 
 About OpenAI is an AI research and deployment company. Our mission is to ensure that artificial general intelligence benefits all of humanity. Our vision for the future of AGI Our mission is to ensure that artificial general intelligence—AI systems that are generally smarter than humans—benefits all of humanity. Read our plan for AGI 
</result>
<result>
 Sam Altman - Wikipedia 
 Samuel H. Altman ( / ˈɔːltmən / AWLT-mən; born April 22, 1985) is an American entrepreneur, investor, and programmer. [2] He is the CEO of OpenAI and the former president of Y Combinator. [3] [4] Altman is also the co-founder of Loopt (founded in 2005) and Worldcoin (founded in 2020). Early life and education [ edit] 
</result> assistant The founders of OpenAI are Sam Altman, Reid Hoffman, Jessica Livingston, Elon Musk, Ilya Sutskever, Peter Thiel and others.

Now let's write the program:

In [ ]:
def is_search(completion):
    return '<search>' in completion
def search(query):
    return top_snippets(query, n=3)

prompt = guidance('''{{#system~}}
You are a helpful assistant.
{{~/system}}
{{#user~}}
From now on, whenever your response depends on any factual information, please search the web by using the function <search>query</search> before responding. I will then paste web results in, and you can respond.
{{~/user}}
{{#assistant~}}
Ok, I will do that. Let's do a practice round
{{~/assistant}}
{{>practice_round}}
{{#user~}}
That was great, now let's do another one.
{{~/user}}
{{#assistant~}}
Ok, I'm ready.
{{~/assistant}}
{{#user~}}
{{user_query}}
{{~/user}}
{{#assistant~}}
{{gen "query" stop="</search>"}}{{#if (is_search query)}}</search>{{/if}}
{{~/assistant}}
{{#if (is_search query)}}
{{#user~}}
Search results: {{#each (search query)}}
<result>
{{this.title}}
{{this.snippet}}
</result>{{/each}}
{{~/user}}
{{#assistant~}}
{{gen "answer"}}
{{~/assistant}}
{{/if}}''')

prompt = prompt(practice_round=practice_round, search=search, is_search=is_search)

In [ ]:
query = "What is Microsoft's stock price right now?"
p1 = prompt(user_query=query)
p1

Stop program system You are a helpful assistant. user From now on, whenever your response depends on any factual information, please search the web by using the function <search>query</search> before responding. I will then paste web results in, and you can respond. assistant Ok, I will do that. Let's do a practice round user Who are the founders of OpenAI? assistant <search>OpenAI founders</search> user Search results:
<result>
OpenAI - Wikipedia
OpenAI systems run on the fifth most powerful supercomputer in the world. [5] [6] [7] The organization was founded in San Francisco in 2015 by Sam Altman, Reid Hoffman, Jessica Livingston, Elon Musk, Ilya Sutskever, Peter Thiel and others, [8] [1] [9] who collectively pledged US$ 1 billion. Musk resigned from the board in 2018 but remained a donor.
</result>
<result>
About - OpenAI
About OpenAI is an AI research and deployment company. Our mission is to ensure that artificial general intelligence benefits all of humanity. Our vision for the future of AGI Our mission is to ensure that artificial general intelligence—AI systems that are generally smarter than humans—benefits all of humanity. Read our plan for AGI
</result>
<result>
Sam Altman - Wikipedia
Samuel H. Altman ( / ˈɔːltmən / AWLT-mən; born April 22, 1985) is an American entrepreneur, investor, and programmer. [2] He is the CEO of OpenAI and the former president of Y Combinator. [3] [4] Altman is also the co-founder of Loopt (founded in 2005) and Worldcoin (founded in 2020). Early life and education [ edit]
</result> assistant The founders of OpenAI are Sam Altman, Reid Hoffman, Jessica Livingston, Elon Musk, Ilya Sutskever, Peter Thiel and others. user That was great, now let's do another one. assistant Ok, I'm ready. user What is Microsoft's stock price right now? assistant <search>Microsoft stock price </search> 
<span style='background-color: rgba(0, 138.56128016, 250.76166089, 0.25); display: inline;' title='&#123;&#123;#if (is_search query)&#125;&#125;
&#123;&#123;#user~&#125;&#125;
Search results: &#123;&#123;#each (search query)&#125;&#125;
<result>
&#123;&#123;this.title&#125;&#125;
&#123;&#123;this.snippet&#125;&#125;
</result>&#123;&#123;/each&#125;&#125;
&#123;&#123;~/user&#125;&#125;
&#123;&#123;#assistant~&#125;&#125;
&#123;&#123;gen "answer"&#125;&#125;
&#123;&#123;~/assistant&#125;&#125;
&#123;&#123;/if&#125;&#125;'> user Search results: 
<result>
 Microsoft Corp (MSFT) Stock Price &amp; News - Google Finance 
 Microsoft Corp (MSFT) Stock Price & News - Google Finance Home MSFT • NASDAQ Microsoft Corp Follow Share $288.37 After Hours: $287.86 (0.18%) -0.51 Closed: Apr 18, 5:57:32 PM GMT-4 ·... 
</result>
<result>
 Microsoft Corporation (MSFT) Stock Price, News, Quote &amp; History - Yahoo ... 
 Microsoft Corporation (MSFT) Stock Price, News, Quote & History - Yahoo Finance U.S. markets closed -4.31 Russell 2000 -7.29(-0.40%) (+0.05%) -2.80 HAPPENING SOON: Yahoo Finance breaks... 
</result>
<result>
 Microsoft Corporation (MSFT) Stock Price, Quote, News &amp; History - Nasdaq 
 Microsoft Corporation Common Stock (MSFT) Stock Quotes - Nasdaq offers stock quotes & market activity data for US and global markets. 
</result> assistant Microsoft's stock price is currently $288.37. Please note that stock prices are constantly changing, so it's best to check an updated source for the most accurate information.

In [ ]:
query = "Who is Marco Tulio Ribeiro?"
p2 = prompt(user_query=query)
p2



Stop program system You are a helpful assistant. user From now on, whenever your response depends on any factual information, please search the web by using the function <search>query</search> before responding. I will then paste web results in, and you can respond. assistant Ok, I will do that. Let's do a practice round user Who are the founders of OpenAI? assistant <search>OpenAI founders</search> user Search results:
<result>
OpenAI - Wikipedia
OpenAI systems run on the fifth most powerful supercomputer in the world. [5] [6] [7] The organization was founded in San Francisco in 2015 by Sam Altman, Reid Hoffman, Jessica Livingston, Elon Musk, Ilya Sutskever, Peter Thiel and others, [8] [1] [9] who collectively pledged US$ 1 billion. Musk resigned from the board in 2018 but remained a donor.
</result>
<result>
About - OpenAI
About OpenAI is an AI research and deployment company. Our mission is to ensure that artificial general intelligence benefits all of humanity. Our vision for the future of AGI Our mission is to ensure that artificial general intelligence—AI systems that are generally smarter than humans—benefits all of humanity. Read our plan for AGI
</result>
<result>
Sam Altman - Wikipedia
Samuel H. Altman ( / ˈɔːltmən / AWLT-mən; born April 22, 1985) is an American entrepreneur, investor, and programmer. [2] He is the CEO of OpenAI and the former president of Y Combinator. [3] [4] Altman is also the co-founder of Loopt (founded in 2005) and Worldcoin (founded in 2020). Early life and education [ edit]
</result> assistant The founders of OpenAI are Sam Altman, Reid Hoffman, Jessica Livingston, Elon Musk, Ilya Sutskever, Peter Thiel and others. user That was great, now let's do another one. assistant Ok, I'm ready. user Who is Marco Tulio Ribeiro? assistant <search>Marco Tulio Ribeiro </search> 
<span style='background-color: rgba(0, 138.56128016, 250.76166089, 0.25); display: inline;' title='&#123;&#123;#if (is_search query)&#125;&#125;
&#123;&#123;#user~&#125;&#125;
Search results: &#123;&#123;#each (search query)&#125;&#125;
<result>
&#123;&#123;this.title&#125;&#125;
&#123;&#123;this.snippet&#125;&#125;
</result>&#123;&#123;/each&#125;&#125;
&#123;&#123;~/user&#125;&#125;
&#123;&#123;#assistant~&#125;&#125;
&#123;&#123;gen "answer"&#125;&#125;
&#123;&#123;~/assistant&#125;&#125;
&#123;&#123;/if&#125;&#125;'> user Search results: 
<result>
 Marco Tulio Ribeiro - University of Washington 
 Marco Tulio Ribeiro – I'm a researcher at Microsoft Research, in the Adaptive Systems and Interaction group. I am also an Affiliate Assistant Professor at the University of Washington, where I was previously a Ph.D student advised by Carlos Guestrin and Sameer Singh. 
</result>
<result>
 Marco Tulio Ribeiro at Microsoft Research 
 Marco Tulio Ribeiro Senior Researcher About Projects Publications News & features About My research focus is helping humans interact with machine learning models meaningfully. That involves interpretability, trust, debugging, feedback, etc. Check out my personal webpage for more details. Research groups Adaptive Systems and Interaction Group 
</result>
<result>
 ‪Marco Tulio Ribeiro‬ - ‪Google Scholar‬ 
 Marco Tulio Ribeiro. Microsoft Research. Verified email at cs.washington.edu - Homepage. Machine Learning Natural Language Processing. Articles Cited by Public access Co-authors. Title. ... MT Ribeiro, N Ziviani, ESD Moura, I Hata, A Lacerda, A Veloso. ACM Transactions on Intelligent Systems and Technology (TIST) 5 (4), 1-20, 2014. 134: 
</result> assistant Marco Tulio Ribeiro is a researcher at Microsoft Research in the Adaptive Systems and Interaction group. He is also an Affiliate Assistant Professor at the University of Washington, where he was previously a Ph.D. student advised by Carlos Guestrin and Sameer Singh. His research focuses on helping humans interact with machine learning models meaningfully, involving interpretability, trust, debugging, feedback, and more.

Let's try a query where it should not search the web

In [ ]:
query = "What is 1+1?"
p3 = prompt(user_query=query)
p3


Stop program system You are a helpful assistant. user From now on, whenever your response depends on any factual information, please search the web by using the function <search>query</search> before responding. I will then paste web results in, and you can respond. assistant Ok, I will do that. Let's do a practice round user Who are the founders of OpenAI? assistant <search>OpenAI founders</search> user Search results:
<result>
OpenAI - Wikipedia
OpenAI systems run on the fifth most powerful supercomputer in the world. [5] [6] [7] The organization was founded in San Francisco in 2015 by Sam Altman, Reid Hoffman, Jessica Livingston, Elon Musk, Ilya Sutskever, Peter Thiel and others, [8] [1] [9] who collectively pledged US$ 1 billion. Musk resigned from the board in 2018 but remained a donor.
</result>
<result>
About - OpenAI
About OpenAI is an AI research and deployment company. Our mission is to ensure that artificial general intelligence benefits all of humanity. Our vision for the future of AGI Our mission is to ensure that artificial general intelligence—AI systems that are generally smarter than humans—benefits all of humanity. Read our plan for AGI
</result>
<result>
Sam Altman - Wikipedia
Samuel H. Altman ( / ˈɔːltmən / AWLT-mən; born April 22, 1985) is an American entrepreneur, investor, and programmer. [2] He is the CEO of OpenAI and the former president of Y Combinator. [3] [4] Altman is also the co-founder of Loopt (founded in 2005) and Worldcoin (founded in 2020). Early life and education [ edit]
</result> assistant The founders of OpenAI are Sam Altman, Reid Hoffman, Jessica Livingston, Elon Musk, Ilya Sutskever, Peter Thiel and others. user That was great, now let's do another one. assistant Ok, I'm ready. user What is 1+1? assistant 1+1 equals 2. 
<span style='background-color: rgba(0, 138.56128016, 250.76166089, 0.25); display: inline;' title='&#123;&#123;#if (is_search query)&#125;&#125;
&#123;&#123;#user~&#125;&#125;
Search results: &#123;&#123;#each (search query)&#125;&#125;
<result>
&#123;&#123;this.title&#125;&#125;
&#123;&#123;this.snippet&#125;&#125;
</result>&#123;&#123;/each&#125;&#125;
&#123;&#123;~/user&#125;&#125;
&#123;&#123;#assistant~&#125;&#125;
&#123;&#123;gen "answer"&#125;&#125;
&#123;&#123;~/assistant&#125;&#125;
&#123;&#123;/if&#125;&#125;'>

<hr style="height: 1px; opacity: 0.5; border: none; background: #cccccc;">
<div style="text-align: center; opacity: 0.5">Have an idea for more helpful examples? Pull requests that add to this documentation notebook are encouraged!</div>